In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
df = pd.read_csv(r'C:\Users\Jouke\Documents\evedata-logger\output\market_data_with_names_merged.csv', parse_dates=['date'])

In [3]:
item_names = sorted(df['type_name'].unique())

In [4]:
dropdown = widgets.Combobox(
    options=item_names,
    placeholder='Search item',
    description='Item:',
    ensure_option=True,
    continuous_update=False
)

In [5]:
ma_slider = widgets.IntSlider(
    value=7,
    min=1,
    max=30,
    step=1,
    description='MA Window',
    continuous_update=False
)

days_slider = widgets.IntSlider(
    value=365,
    min=1,
    max=len(df['date'].unique()),
    step=1,
    description='Days Shown',
    continuous_update=False
)

donchian_slider = widgets.IntSlider(
    value=20,
    min=5,
    max=90,
    step=1,
    description='Donchian Window',
    continuous_update=False
)

checkboxes = {
    "ma": widgets.Checkbox(value=True, description='Show MA', indent=False),
    "donchian": widgets.Checkbox(value=True, description='Show Donchian Channel', indent=False),
    "volume": widgets.Checkbox(value=True, description='Show Volume', indent=False)
}

output = widgets.Output()

In [6]:
def plot_trend(type_name, ma_window, days_shown, donchian_window, ma, donchian, volume):
    with output:
        clear_output(wait=True)
        item_df = df[df['type_name'] == type_name].sort_values('date')
        if item_df.empty:
            print("No data for selected item.")
            return
        item_df = item_df.iloc[-days_shown:]

        fig, ax1 = plt.subplots(figsize=(10, 5))

        # Donchian Channel
        if donchian:
            donchian_high = item_df['highest'].rolling(window=donchian_window).max()
            donchian_low = item_df['lowest'].rolling(window=donchian_window).min()
            ax1.fill_between(item_df['date'], donchian_low, donchian_high, color='cyan', alpha=0.1, label=f'Donchian {donchian_window}')
            ax1.plot(item_df['date'], donchian_high, color='cyan', linestyle='--', label='Donchian High')
            ax1.plot(item_df['date'], donchian_low, color='deepskyblue', linestyle='--', label='Donchian Low')

        # Daily average
        ax1.plot(item_df['date'], item_df['average'], alpha=0.4, label='Daily Average', color='blue')

        # Moving Average
        if ma:
            ax1.plot(item_df['date'], item_df['average'].rolling(window=ma_window).mean(), color='red', label=f'{ma_window}-Day MA')

        ax1.set_xlabel('Date')
        ax1.set_ylabel('Average Price (ISK)', color='blue')
        ax1.tick_params(axis='y', labelcolor='blue')

        # Volume bars
        if volume:
            ax2 = ax1.twinx()
            ax2.bar(item_df['date'], item_df['volume'], width=1.0, color='gray', alpha=0.3, label='Daily Volume')
            ax2.set_ylabel('Volume', color='gray')
            ax2.tick_params(axis='y', labelcolor='gray')

            lines, labels = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax1.legend(lines + lines2, labels + labels2, loc='upper left')
        else:
            ax1.legend(loc='upper left')

        plt.title(f'{type_name} Price, Donchian Channel, & Volume\nLast {days_shown} Days')
        plt.tight_layout()
        plt.show()
        plt.close(fig)      # <-- This is essential

        return None         # <-- Explicitly return nothing


# Group controls for a tidy UI
controls = widgets.VBox([
    dropdown,
    widgets.HBox([ma_slider, donchian_slider, days_slider]),
    widgets.HBox([checkboxes["ma"], checkboxes["donchian"], checkboxes["volume"]])
])

interactive_plot = widgets.interactive_output(
    plot_trend, {
        'type_name': dropdown,
        'ma_window': ma_slider,
        'days_shown': days_slider,
        'donchian_window': donchian_slider,
        'ma': checkboxes["ma"],
        'donchian': checkboxes["donchian"],
        'volume': checkboxes["volume"]
    }
)

display(controls, output)



Output()